In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [5]:
tf.test.gpu_device_name()

''

# 2.3 自动求梯度


在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用 tensorflow2.0 提供的 GradientTape 来自动求梯度。

## 2.3.1 简单示例

我们先看一个简单例子：对函数  

$y=2x^Tx$

求关于列向量 $x$ 的梯度。我们先创建变量 $x$，并赋初值。

In [6]:
x = tf.reshape(tf.Variable(range(4), dtype=tf.float32), (-1,1))

In [7]:
x

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

In [8]:
tf.transpose(x)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 1., 2., 3.]], dtype=float32)>

In [9]:
tf.matmul(tf.transpose(x) , x)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[14.]], dtype=float32)>

In [10]:
x = tf.reshape(tf.Variable(range(4), dtype=tf.float32), (4,1))

In [11]:
x

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

In [12]:
tf.transpose(x)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 1., 2., 3.]], dtype=float32)>

In [13]:
tf.matmul(tf.transpose(x) , x)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[14.]], dtype=float32)>

函数 $y=2x^Tx$ 关于 $x$ 的梯度应为 $4x$。现在我们来验证一下求出来的梯度是正确的。

In [14]:
with tf.GradientTape() as t:
    t.watch(x)
    y = 2 * tf.matmul(tf.transpose(x), x)

In [15]:
dy_dx = t.gradient(y,x)

In [16]:
dy_dx

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

# 2.3.2 训练模式和预测模式

In [17]:
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x*x
    z = y*y
    dz_dx = g.gradient(z, x)
    dy_dx = g.gradient(y, x)
    
dz_dx,dy_dx

(<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[  0.],
        [  4.],
        [ 32.],
        [108.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[0.],
        [2.],
        [4.],
        [6.]], dtype=float32)>)

# 2.3.3 对Python控制流求梯度

即使函数的计算图包含了 Python 的控制流（如条件和循环控制），我们也有可能对变量求梯度。

考虑下面程序，其中包含 Python 的条件和循环控制。

需要强调的是，这里循环（while循环）迭代的次数和条件判断（if语句）的执行都取决于输入 a 的值。

In [18]:
def f(a):
    b = a * 2
    while tf.norm(b) < 1000:
        b *= 2
    if tf.reduce_sum(b) > 0:
        c = b
    else :
        c = 100 * b

    return c

我们来分析一下上面定义的 $f$ 函数。

事实上，给定任意输入 $a$ ，其输出必然是 $f(a) = x * a$ 的形式，其中标量系数 $x$ 的值取决于输入 $a$。

由于 $c = f(a)$ 有关 $a$ 的梯度为 $x$ ，且值为 $c / a$，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性。

In [19]:
a = tf.random.normal((1,1),dtype=tf.float32)
with tf.GradientTape() as t:
    t.watch(a)
    c = f(a)
t.gradient(c,a) == c/a

<tf.Tensor: shape=(1, 1), dtype=bool, numpy=array([[ True]])>